In [1]:
from pystac_client import Client
from odc.stac import load
from dask.distributed import Client as DaskClient


from planetary_computer import sign_url
import numpy as np
import warnings
import folium

In [2]:
# Microsoft Planetary Computer STAC Catalog URL
catalog = "https://planetarycomputer.microsoft.com/api/stac/v1"

# Create a STAC Client
client = Client.open(catalog)

# Parameters for the query
collection = "sentinel-1-rtc"
datetime = "2024-01/2024-03"  # 3 months

In [9]:
# Use a bbox over Singapore Strait
bbox = [103.80, 1.10, 104.00, 1.30]

In [ ]:
items = client.search(
    collections=["sentinel-1-rtc"],
    # intersects=geom,
    bbox=bbox,
    datetime=datetime
).item_collection()

print(f"Found {len(items)} items")

In [ ]:
data = load(
    items,
    # geopolygon=geom,
    bbox=bbox,
    measurements=["vv", "vh"],
    groupby="solar_day",
    chunks={"x": 2048, "y": 2048},
    patch_url=sign_url,
)

# Convert to decibels
data["vv_db"] = 10 * np.log10(data.vv)
data["vh_db"] = 10 * np.log10(data.vh)

data

In [ ]:
# Supress the RuntimeWarning for log10(0)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# This step will take about three minutes to process
with DaskClient(n_workers=2, threads_per_worker=16) as client:
    vv_max = data.vv_db.max(dim=["time"]).compute()
    vh_max = data.vh_db.max(dim=["time"]).compute()

In [ ]:
m = folium.Map()

vv_max.odc.add_to(m, cmap="viridis", name="VV max")
vh_max.odc.add_to(m, cmap="viridis", name="VH max")

folium.LayerControl().add_to(m)
m.fit_bounds(vv_max.odc.map_bounds())
m